### Necessary Libraries Import

In [1]:
import os
import numpy as np
import pandas as pd
import shutil

# The path can also be read from a config file, etc.
OPENSLIDE_PATH = r'E:\KSA Project\data_preprocessing\openslide-bin-4.0.0.3-windows-x64\bin'

import os
if hasattr(os, 'add_dll_directory'):
    # Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

### WSI DownSampling and Resizing

In [2]:
# define some configureable parameters here
SOURCE_DIR = r'E:\KSA Project\dataset\svs_files'
DOWNSAMPLE_IMG_DIR = r'E:\KSA Project\dataset\downsample_crop'
CROP_IMG_DIR = r'E:\KSA Project\dataset\downsample_crop'

In [4]:
import os
from WSI_Stiching_Code.wsi_core.WholeSlideImage import WholeSlideImage
from WSI_Stiching_Code.Image_Crop import crop_downsample_image
import openslide

# Set OpenSlide Path
OPENSLIDE_PATH = r'E:\KSA Project\data_preprocessing\openslide-bin-4.0.0.3-windows-x64\bin'
if hasattr(os, 'add_dll_directory'):
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

def get_wsi_magnification(slide):
    """
    Retrieve the magnification power of the whole slide image (WSI).
    """
    try:
        magnification = slide.properties[openslide.PROPERTY_NAME_OBJECTIVE_POWER]
        return float(magnification)
    except KeyError:
        raise ValueError("Magnification information not available in the WSI properties.")

def save_downsampled_cropped_image(WSI_object, slide_id, save_dir, seg_level):
    """
    Save a downsampled and cropped version of the whole slide image.
    """
    downsampled_image = WSI_object.getOpenSlide().read_region((0, 0), seg_level, WSI_object.level_dim[seg_level])
    downsampled_image = downsampled_image.convert("RGB")
    # print dimensions of downsampled image 
    print(f"Downsampled image dimensions: {downsampled_image.size}")
    cropped_save_path = os.path.join(save_dir, slide_id + '.png')
    crop_downsample_image(downsampled_image, cropped_save_path)

def process_slide(slide_path, save_dir, target_magnification, crop_size):
    """
    Process a single slide for downsampling and cropping.
    """
    slide_id = os.path.splitext(os.path.basename(slide_path))[0]

    try:
        WSI_object = WholeSlideImage(slide_path)
        original_magnification = get_wsi_magnification(WSI_object.getOpenSlide())
        print(f"Original dimensions: {WSI_object.level_dim[0]} of slide {slide_id}")
        scale_factor = original_magnification / target_magnification
        best_level = WSI_object.getOpenSlide().get_best_level_for_downsample(scale_factor)

        save_downsampled_cropped_image(WSI_object, slide_id, save_dir, best_level)

    except Exception as e:
        print(f"Error processing slide {slide_id}: {e}")

# Configurable Parameters
source_dir = SOURCE_DIR # Path to the directory containing WSI files
save_dir = DOWNSAMPLE_IMG_DIR  # Path to save the processed images
target_magnification = 5  # Target magnification for downsampling
crop_size = 512  # Crop size for the output images

os.makedirs(save_dir, exist_ok=True)

# Process each slide in the source directory
slides = [os.path.join(source_dir, slide) for slide in os.listdir(source_dir) if slide.endswith(('.svs', '.tiff'))]

for slide_path in slides:
    process_slide(slide_path, save_dir, target_magnification, crop_size)


(1.0, 1.0) ESTIMATED DOWNSAMPLE 
(4.001378043178686, 4.0) ESTIMATED DOWNSAMPLE 
ASSERT LEVEL DOWSSAMPLE: 
dim[0] (8711, 10000)
LVL downsample  [(1.0, 1.0), (4.001378043178686, 4.0)]
Error processing slide TCGA-AA-3489_nonMSIH: Magnification information not available in the WSI properties.
(1.0, 1.0) ESTIMATED DOWNSAMPLE 
(4.000887311446317, 4.0) ESTIMATED DOWNSAMPLE 
ASSERT LEVEL DOWSSAMPLE: 
dim[0] (9018, 10000)
LVL downsample  [(1.0, 1.0), (4.000887311446317, 4.0)]
Error processing slide TCGA-AA-3492_MSIH: Magnification information not available in the WSI properties.
(1.0, 1.0) ESTIMATED DOWNSAMPLE 
(4.0, 4.0) ESTIMATED DOWNSAMPLE 
(16.0, 16.0) ESTIMATED DOWNSAMPLE 
(32.0, 32.0) ESTIMATED DOWNSAMPLE 
ASSERT LEVEL DOWSSAMPLE: 
dim[0] (82688, 81664)
LVL downsample  [(1.0, 1.0), (4.0, 4.0), (16.0, 16.0), (32.0, 32.0)]
Original dimensions: (82688, 81664) of slide TCGA-AA-3667-01Z-00-DX1
Downsampled image dimensions: (20672, 20416)
Cropped image dimensions: (4627, 4494)
(1.0, 1.0) ESTIMA

### Save WSI as JPEG or PNG

In [3]:
import os
from PIL import Image

def save_full_resolution_svs_as_image(svs_path, output_dir, output_format="JPEG"):
    """
    Save the full-resolution WSI image as a JPEG or PNG image without downscaling.

    Parameters:
        svs_path (str): Path to the .svs file.
        output_dir (str): Directory to save the converted image.
        output_format (str): Format to save the image ("JPEG" or "PNG").
    """
    try:
        # Load the slide
        slide = openslide.OpenSlide(svs_path)

        # Get the full dimensions (width, height) of the slide
        width, height = slide.dimensions
        print(f"Image Dimensions: {width} x {height} pixels")
        # Read the full-resolution region (entire image)
        full_image = slide.read_region((0, 0), 0, (width, height)).convert("RGB")

        # Create the output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

        # Construct the output file path
        slide_filename = os.path.splitext(os.path.basename(svs_path))[0]
        output_path = os.path.join(output_dir, f"{slide_filename}.{output_format.lower()}")

        # Save the image
        full_image.save(output_path, format=output_format)
        print(f"Saved full-resolution image: {output_path}")
        print(f"Image Dimensions: {width} x {height} pixels\n")
    except MemoryError:
        print(f"Error: File too large to save at full resolution for {svs_path}")
    except Exception as e:
        print(f"Error saving image {svs_path}: {e}")


def process_svs_to_images(input_dir, output_dir, output_format="JPEG"):
    """
    Process all .svs files in a directory and save them as full-resolution images.

    Parameters:
        input_dir (str): Directory containing .svs files.
        output_dir (str): Directory to save the converted images.
        output_format (str): Format to save the images ("JPEG" or "PNG").
    """
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".svs"):
                svs_path = os.path.join(root, file)
                save_full_resolution_svs_as_image(svs_path, output_dir, output_format)


# Example usage
input_dir = r"E:\KSA Project\dataset\svs_files"  # Path to .svs files
output_dir = r"E:\KSA Project\dataset\converted_images"  # Directory to save images
process_svs_to_images(input_dir, output_dir, output_format="JPEG")


Image Dimensions: 8711 x 10000 pixels
Saved full-resolution image: E:\KSA Project\dataset\converted_images\TCGA-AA-3489_nonMSIH.jpeg
Image Dimensions: 8711 x 10000 pixels

Image Dimensions: 9018 x 10000 pixels
Saved full-resolution image: E:\KSA Project\dataset\converted_images\TCGA-AA-3492_MSIH.jpeg
Image Dimensions: 9018 x 10000 pixels

Image Dimensions: 30695 x 38187 pixels
Saved full-resolution image: E:\KSA Project\dataset\converted_images\TCGA-AD-5900_MSIH.jpeg
Image Dimensions: 30695 x 38187 pixels

Image Dimensions: 10000 x 6751 pixels
Saved full-resolution image: E:\KSA Project\dataset\converted_images\TCGA-AG-3592_nonMSIH.jpeg
Image Dimensions: 10000 x 6751 pixels



In [6]:
import os
from WSI_Stiching_Code.wsi_core.WholeSlideImage import WholeSlideImage
from WSI_Stiching_Code.Image_Crop import crop_downsample_image
import openslide

# Set OpenSlide Path
OPENSLIDE_PATH = r'E:\KSA Project\data_preprocessing\openslide-bin-4.0.0.3-windows-x64\bin'
if hasattr(os, 'add_dll_directory'):
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

def get_wsi_magnification(slide):
    try:
        magnification = slide.properties[openslide.PROPERTY_NAME_OBJECTIVE_POWER]
        return float(magnification)
    except KeyError:
        raise ValueError("Magnification information not available in the WSI properties.")

def save_downsampled_cropped_image(WSI_object, slide_id, save_dir, seg_level):
    downsampled_image = WSI_object.getOpenSlide().read_region((0, 0), seg_level, WSI_object.level_dim[seg_level])
    downsampled_image = downsampled_image.convert("RGB")
    # print dimensions of downsampled image 
    print(f"Downsampled image dimensions: {downsampled_image.size}")

def process_slide(slide_path, save_dir, target_magnification):
    slide_id = os.path.splitext(os.path.basename(slide_path))[0]
    try:
        WSI_object = WholeSlideImage(slide_path)
        original_magnification = get_wsi_magnification(WSI_object.getOpenSlide())
        print(f'\n Magnification: {original_magnification}')
        print(f"Original dimensions: {WSI_object.level_dim[0]} of slide {slide_id}")
        scale_factor = original_magnification / 5
        best_level = WSI_object.getOpenSlide().get_best_level_for_downsample(scale_factor)
        print(f'Best Level: {best_level} by scale factor: {scale_factor}')
        save_downsampled_cropped_image(WSI_object, slide_id, save_dir, best_level)
    except Exception as e:
        print(f"Error processing slide {slide_id}: {e}")

# Configurable Parameters
source_dir = SOURCE_DIR # Path to the directory containing WSI files
save_dir = DOWNSAMPLE_IMG_DIR  # Path to save the processed images
target_magnification = 5  # Target magnification for downsampling
# os.makedirs(save_dir, exist_ok=True)

# Process each slide in the source directory
slides = [os.path.join(source_dir, slide) for slide in os.listdir(source_dir) if slide.endswith(('.svs', '.tiff'))]

for slide_path in slides:
    process_slide(slide_path, save_dir, target_magnification)



 Magnification: 40.0
Original dimensions: (95615, 74462) of slide TCGA-3L-AA1B_nonMSIH
Best Level: 1 by scale factor: 8.0
Downsampled image dimensions: (23903, 18615)

 Magnification: 40.0
Original dimensions: (101680, 36748) of slide TCGA-A6-2671_nonMSIH
Best Level: 1 by scale factor: 8.0
Downsampled image dimensions: (25420, 9187)

 Magnification: 40.0
Original dimensions: (157316, 21227) of slide TCGA-A6-2675_nonMSIH
Best Level: 1 by scale factor: 8.0
Downsampled image dimensions: (39329, 5306)

 Magnification: 40.0
Original dimensions: (151560, 46538) of slide TCGA-A6-3807_nonMSIH
Best Level: 1 by scale factor: 8.0
Downsampled image dimensions: (37890, 11634)
Error processing slide TCGA-AA-3489_nonMSIH: Magnification information not available in the WSI properties.
Error processing slide TCGA-AA-3492_MSIH: Magnification information not available in the WSI properties.

 Magnification: 20.0
Original dimensions: (82688, 81664) of slide TCGA-AA-3667-01Z-00-DX1
Best Level: 1 by scale 